In [1]:
import os
import shutil
from sklearn.model_selection import KFold

# Diretórios que contém as pastas com os arquivos .npy
bases =[ '../input/10',
         '../input/20', 
         '../input/30', 
         '../input/40',
         '../label/10',
         '../label/20',
         '../label/30',
         '../label/40',
       
        ]

for base_dir in bases:
        # Pastas a serem processadas
        folders = ['ammonia', 'bikesharing', 'daily', 'distillate', 'electricity', 'lds', 'sunspots']

        # Diretório onde as pastas de folds serão criadas
        output_dir = os.path.join(base_dir, 'folds')
        os.makedirs(output_dir, exist_ok=True)

        # Função para distribuir arquivos
        def distribute_files(files, fold_num, train_idx, val_idx, test_idx, fold_dir):
            for i, filename in enumerate(files):
                if i in train_idx:
                    dest_folder = os.path.join(fold_dir, 'train')
                elif i in val_idx:
                    dest_folder = os.path.join(fold_dir, 'val')
                else:
                    dest_folder = os.path.join(fold_dir, 'test')
                
                os.makedirs(dest_folder, exist_ok=True)
                shutil.copy2(os.path.join(base_dir, folder, filename), os.path.join(dest_folder, filename))

        # Processar cada pasta
        for folder in folders:
            folder_path = os.path.join(base_dir, folder)
            files = sorted([f for f in os.listdir(folder_path) if f.endswith('.npy')])
            
            num_files = len(files)
            num_folds = min(5, num_files)  # Ajuste o número de folds para não exceder o número de arquivos

            if num_files < 3:
                # Exceções para pastas com menos de 3 arquivos
                for fold_num in range(1, 6):
                    fold_dir = os.path.join(output_dir, f'fold{fold_num}')
                    os.makedirs(fold_dir, exist_ok=True)
                    os.makedirs(os.path.join(fold_dir, 'train'), exist_ok=True)
                    os.makedirs(os.path.join(fold_dir, 'val'), exist_ok=True)
                    os.makedirs(os.path.join(fold_dir, 'test'), exist_ok=True)
                    
                    if num_files == 1:
                        # Todos os arquivos vão para o teste
                        shutil.copy2(os.path.join(folder_path, files[0]), os.path.join(fold_dir, 'test', files[0]))
                    elif num_files == 2:
                        # Um arquivo para treino e outro para teste
                        shutil.copy2(os.path.join(folder_path, files[0]), os.path.join(fold_dir, 'train', files[0]))
                        shutil.copy2(os.path.join(folder_path, files[1]), os.path.join(fold_dir, 'test', files[1]))
                    elif num_files == 3:
                        # Um arquivo para cada (treino, validação, teste)
                        shutil.copy2(os.path.join(folder_path, files[0]), os.path.join(fold_dir, 'train', files[0]))
                        shutil.copy2(os.path.join(folder_path, files[1]), os.path.join(fold_dir, 'val', files[1]))
                        shutil.copy2(os.path.join(folder_path, files[2]), os.path.join(fold_dir, 'test', files[2]))
            else:
                # Divisão para pastas com 3 ou mais arquivos
                kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
                
                for fold_num, (train_val_idx, test_idx) in enumerate(kf.split(files), 1):
                    train_size = int(0.7 * len(train_val_idx))
                    train_idx = train_val_idx[:train_size]
                    val_idx = train_val_idx[train_size:]
                    
                    fold_dir = os.path.join(output_dir, f'fold{fold_num}')
                    os.makedirs(fold_dir, exist_ok=True)
                    os.makedirs(os.path.join(fold_dir, 'train'), exist_ok=True)
                    os.makedirs(os.path.join(fold_dir, 'val'), exist_ok=True)
                    os.makedirs(os.path.join(fold_dir, 'test'), exist_ok=True)
                    
                    distribute_files(files, fold_num, train_idx, val_idx, test_idx, fold_dir)

        print(f"Divisão de {base_dir} concluída com sucesso!")


Divisão de ../input/10 concluída com sucesso!
Divisão de ../input/20 concluída com sucesso!
Divisão de ../input/30 concluída com sucesso!
Divisão de ../input/40 concluída com sucesso!
Divisão de ../label/10 concluída com sucesso!
Divisão de ../label/20 concluída com sucesso!
Divisão de ../label/30 concluída com sucesso!
Divisão de ../label/40 concluída com sucesso!


In [2]:
import os
import shutil

# Diretórios base
input_base = r'../input'
label_base = r'../label'

# Função para deletar subpastas dentro de cada diretório principal
def delete_non_fold_subfolders(base_dir):
    if not os.path.exists(base_dir):
        print(f'O diretório {base_dir} não existe. Pulando...')
        return

    # Percorre os diretórios dentro de input/ ou label/
    for main_folder in os.listdir(base_dir):  # Ex: ['10', '20', '30', '40']
        main_folder_path = os.path.join(base_dir, main_folder)

        if not os.path.isdir(main_folder_path):
            continue  # Se não for diretório, ignora
        
        # Agora dentro de cada diretório (ex: input/10/), verifica as subpastas
        for subfolder in os.listdir(main_folder_path):
            subfolder_path = os.path.join(main_folder_path, subfolder)

            if os.path.isdir(subfolder_path) and 'fold' not in subfolder.lower():
                print(f'Deletando diretório: {subfolder_path}')
                shutil.rmtree(subfolder_path)  # Deleta apenas subpastas que não são "folds"
            else:
                print(f'Mantendo diretório: {subfolder_path}')

# Aplicar função nos diretórios de input e label
delete_non_fold_subfolders(input_base)
delete_non_fold_subfolders(label_base)

print('Processo concluído!')


Deletando diretório: ../input\10\ammonia
Deletando diretório: ../input\10\bikesharing
Deletando diretório: ../input\10\daily
Deletando diretório: ../input\10\distillate
Deletando diretório: ../input\10\electricity
Mantendo diretório: ../input\10\folds
Deletando diretório: ../input\10\lds
Deletando diretório: ../input\10\sunspots
Deletando diretório: ../input\20\ammonia
Deletando diretório: ../input\20\bikesharing
Deletando diretório: ../input\20\daily
Deletando diretório: ../input\20\distillate
Deletando diretório: ../input\20\electricity
Mantendo diretório: ../input\20\folds
Deletando diretório: ../input\20\lds
Deletando diretório: ../input\20\sunspots
Deletando diretório: ../input\30\ammonia
Deletando diretório: ../input\30\bikesharing
Deletando diretório: ../input\30\daily
Deletando diretório: ../input\30\distillate
Deletando diretório: ../input\30\electricity
Mantendo diretório: ../input\30\folds
Deletando diretório: ../input\30\lds
Deletando diretório: ../input\30\sunspots
Deletan

In [3]:
import os
import shutil

# Caminho principal onde estão as pastas de input e index
main_path = r"../input"
main_index = r"../index"

# Lista de prefixos das pastas, para mapear corretamente
valid_prefixes = {
    "ammonia": "am",
    "bikesharing": "bi",
    "daily": "da",
    "distillate": "di",
    "electricity": "el",
    "lds": "mt",
    "sunspots": "su"
}

# Listar todas as pastas de "test" dentro de cada "foldX"
test_folders = [os.path.join(root, d) for root, dirs, _ in os.walk(main_path) for d in dirs if "test" in d.lower()]

# Processar cada pasta "test"
for test_folder in test_folders:
    # Identificar a pasta "foldX" e criar pasta "index" dentro dela
    fold_path = os.path.dirname(test_folder)
    index_folder = os.path.join(fold_path, "index")
    os.makedirs(index_folder, exist_ok=True)

    # Pegar o número da base (10, 20, 30, 40, etc.)
    path_parts = test_folder.split(os.sep)
    if len(path_parts) < 3:
        print(f"⚠️ Estrutura inesperada: {test_folder}")
        continue
    base_number = path_parts[1]  # Pega "10", "20", etc.

    # Caminho da pasta de índices correspondente
    index_source_folder = os.path.join(main_index, base_number)
    if not os.path.exists(index_source_folder):
        print(f"⚠️ Pasta de índice não encontrada: {index_source_folder}")
        continue

    # Listar arquivos dentro da pasta test (os arquivos de teste)
    test_files = [f for f in os.listdir(test_folder) if f.endswith(".npy")]

    # Copiar os arquivos de índice correspondentes
    for file_name in test_files:
        file_prefix = file_name[:2].lower()

        # Encontrar a pasta correspondente no índice
        folder_name = next((k for k, v in valid_prefixes.items() if v == file_prefix), None)
        if not folder_name:
            print(f"⚠️ Prefixo não reconhecido: {file_prefix}")
            continue

        matching_folder = os.path.join(index_source_folder, folder_name)
        source_file = os.path.join(matching_folder, file_name)
        dest_file = os.path.join(index_folder, file_name)

        if os.path.exists(source_file):
            shutil.copy2(source_file, dest_file)
            # print(f"✅ Copiado: {source_file} ➝ {dest_file}")
        else:
            print(f"⚠️ Arquivo não encontrado: {source_file}")

print("🎯 Processo concluído!")


🎯 Processo concluído!


In [4]:
import os
import shutil

# Caminho principal onde estão as pastas de input e index
main_path = r"../input"
main_index = r"../index"

# Lista de prefixos das pastas, para mapear corretamente
valid_prefixes = {
    "ammonia": "am",
    "bikesharing": "bi",
    "daily": "da",
    "distillate": "di",
    "electricity": "el",
    "lds": "mt",
    "sunspots": "su"
}

# Listar todas as pastas de "test" dentro de cada "foldX"
test_folders = [os.path.join(root, d) for root, dirs, _ in os.walk(main_path) for d in dirs if "train" in d.lower()]

# Processar cada pasta "test"
for test_folder in test_folders:
    # Identificar a pasta "foldX" e criar pasta "index" dentro dela
    fold_path = os.path.dirname(test_folder)
    index_folder = os.path.join(fold_path, "index_train")
    os.makedirs(index_folder, exist_ok=True)

    # Pegar o número da base (10, 20, 30, 40, etc.)
    path_parts = test_folder.split(os.sep)
    if len(path_parts) < 3:
        print(f"⚠️ Estrutura inesperada: {test_folder}")
        continue
    base_number = path_parts[1]  # Pega "10", "20", etc.

    # Caminho da pasta de índices correspondente
    index_source_folder = os.path.join(main_index, base_number)
    if not os.path.exists(index_source_folder):
        print(f"⚠️ Pasta de índice não encontrada: {index_source_folder}")
        continue

    # Listar arquivos dentro da pasta test (os arquivos de teste)
    test_files = [f for f in os.listdir(test_folder) if f.endswith(".npy")]

    # Copiar os arquivos de índice correspondentes
    for file_name in test_files:
        file_prefix = file_name[:2].lower()

        # Encontrar a pasta correspondente no índice
        folder_name = next((k for k, v in valid_prefixes.items() if v == file_prefix), None)
        if not folder_name:
            print(f"⚠️ Prefixo não reconhecido: {file_prefix}")
            continue

        matching_folder = os.path.join(index_source_folder, folder_name)
        source_file = os.path.join(matching_folder, file_name)
        dest_file = os.path.join(index_folder, file_name)

        if os.path.exists(source_file):
            shutil.copy2(source_file, dest_file)
            # print(f"✅ Copiado: {source_file} ➝ {dest_file}")
        else:
            print(f"⚠️ Arquivo não encontrado: {source_file}")

print("🎯 Processo concluído!")


🎯 Processo concluído!


In [5]:
import os
import shutil

# Caminho principal onde estão as pastas de input e index
main_path = r"../input"
main_index = r"../index"

# Lista de prefixos das pastas, para mapear corretamente
valid_prefixes = {
    "ammonia": "am",
    "bikesharing": "bi",
    "daily": "da",
    "distillate": "di",
    "electricity": "el",
    "lds": "mt",
    "sunspots": "su"
}

# Listar todas as pastas de "test" dentro de cada "foldX"
test_folders = [os.path.join(root, d) for root, dirs, _ in os.walk(main_path) for d in dirs if "val" in d.lower()]

# Processar cada pasta "test"
for test_folder in test_folders:
    # Identificar a pasta "foldX" e criar pasta "index" dentro dela
    fold_path = os.path.dirname(test_folder)
    index_folder = os.path.join(fold_path, "index_val")
    os.makedirs(index_folder, exist_ok=True)

    # Pegar o número da base (10, 20, 30, 40, etc.)
    path_parts = test_folder.split(os.sep)
    if len(path_parts) < 3:
        print(f"⚠️ Estrutura inesperada: {test_folder}")
        continue
    base_number = path_parts[1]  # Pega "10", "20", etc.

    # Caminho da pasta de índices correspondente
    index_source_folder = os.path.join(main_index, base_number)
    if not os.path.exists(index_source_folder):
        print(f"⚠️ Pasta de índice não encontrada: {index_source_folder}")
        continue

    # Listar arquivos dentro da pasta test (os arquivos de teste)
    test_files = [f for f in os.listdir(test_folder) if f.endswith(".npy")]

    # Copiar os arquivos de índice correspondentes
    for file_name in test_files:
        file_prefix = file_name[:2].lower()

        # Encontrar a pasta correspondente no índice
        folder_name = next((k for k, v in valid_prefixes.items() if v == file_prefix), None)
        if not folder_name:
            print(f"⚠️ Prefixo não reconhecido: {file_prefix}")
            continue

        matching_folder = os.path.join(index_source_folder, folder_name)
        source_file = os.path.join(matching_folder, file_name)
        dest_file = os.path.join(index_folder, file_name)

        if os.path.exists(source_file):
            shutil.copy2(source_file, dest_file)
            # print(f"✅ Copiado: {source_file} ➝ {dest_file}")
        else:
            print(f"⚠️ Arquivo não encontrado: {source_file}")

print("🎯 Processo concluído!")


🎯 Processo concluído!
